In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 14.2MB/s 
     |████████████████████████████████| 901kB 54.8MB/s 
     |████████████████████████████████| 3.3MB 60.7MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torch.utils.data import Dataset
import yaml
import torch
import pandas as pd
import torch
from torch.utils.data import random_split
from transformers import TrainingArguments, Trainer, GPTNeoForCausalLM
from transformers import GPT2TokenizerFast as GPT2Tokenizer
import yaml
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re

In [ ]:
with open('/content/drive/MyDrive/data/config.yml', 'r') as f:
    config = yaml.safe_load(f)
torch.cuda.empty_cache()

tokenizer = GPT2Tokenizer.from_pretrained(config['model']['model_name'], bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPTNeoForCausalLM.from_pretrained(config['model']['model_name'])
model.resize_token_embeddings(len(tokenizer))

"""Limpando Cache e Designando Variáveis Aleatórias"""
torch.cuda.empty_cache()

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


In [ ]:
!nvidia-smi

Sat Jun 19 13:22:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    23W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
class ShapingDataset(Dataset):

    def __init__(self, texts):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_len = config['model']['max_seq_length']
        self.input_ids = []
        self.attention_masks = []
        self.labels = []
        for txt in tqdm(texts):
            input_ids, attention_mask = re.findall('\[([ ,0-9]+)\]', txt)
            dici = {'input_ids': [int(x) for x in re.split(', ',input_ids)], 'attention_mask': [int(x) for x in re.split(', ', attention_mask)]}
            self.input_ids.append(torch.tensor(dici['input_ids']))
            self.attention_masks.append(torch.tensor(dici['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_masks[idx]

In [ ]:
tokenized_data = pd.read_csv(config['data']['data'], compression='gzip')
train, test = train_test_split(tokenized_data, test_size=0.15, shuffle=True, random_state=SEED)
train = ShapingDataset(train['texto_tokenizado'])
test = ShapingDataset(test['texto_tokenizado'])

100%|██████████| 40127/40127 [00:48<00:00, 822.28it/s]


In [ ]:
model.to(device)

In [ ]:
training_args = TrainingArguments(output_dir=config['data']['output_dir'],
                                  num_train_epochs=config['model']['num_epochs'],
                                  logging_steps=config['data']['logging_steps'],
                                  save_steps=config['data']['save_steps'],
                                  per_device_train_batch_size=config['model']['batch_size'],
                                  per_device_eval_batch_size=config['model']['batch_size'],                                   
                                  logging_dir=config['data']['logging_dir'],
                                 learning_rate=float(config['model']['learning_rate']))

In [ ]:
trainer = Trainer(model=model, args=training_args,  
                  train_dataset=train,
                  eval_dataset=test,
               data_collator=lambda data: 
              {'input_ids': torch.stack([f[0] for f in data]),       
               'attention_mask': torch.stack([f[1] for f in data]),
               'labels': torch.stack([f[0] for f in data])})

In [ ]:
trainer.train('/content/drive/MyDrive/data/results/checkpoint-175000')

Step,Training Loss
180000,1.368600
185000,1.358500
190000,1.376700
195000,1.360200
200000,1.345800
205000,1.356600
210000,1.346400
215000,1.347800
220000,1.350000
225000,1.345400


TrainOutput(global_step=227382, training_loss=0.312044364674933, metrics={'train_runtime': 12806.7384, 'train_samples_per_second': 17.755, 'train_steps_per_second': 17.755, 'total_flos': 3.4981577779838976e+17, 'train_loss': 0.312044364674933, 'epoch': 1.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/data/small-gpt-neo-finetuned-pt')